In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import optimize

from sklearn.metrics import accuracy_score, f1_score
from pymoo.core.problem import Problem
from evoman.environment import Environment
from fitness_functions import original_fitness, individual_gain
from utils import simulation, verify_solution, init_env, run_pymoo_algorithm, initialise_script
import math


pygame 2.5.1 (SDL 2.28.2, Python 3.9.17)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
solutions = []
for file in os.listdir('solutions_beats_5_enemies'):
    if file.startswith('pymoo'):
        new_solution = []
        with open(f'solutions_beats_5_enemies/{file}') as f:
            solutions.append(f.read().splitlines())
            
solutions = np.array(solutions, dtype=float)

In [3]:
seed = 42
POP_SIZE = 1
ENEMIES = [1, 2, 3, 4, 5, 6, 7, 8]
n_hidden_neurons = 10


In [4]:
class objectives(Problem):
    enemies: list[int]
    env: Environment

    def __init__(self, env: Environment, n_genes: int, enemies: list[int], n_objectives):
        self.env = env
        self.enemies = enemies
        super().__init__(n_var=n_genes, n_obj=n_objectives, xl=-1, xu=1, type_var=float)

    def _evaluate(self, x: list[np.array], out, *args, **kwargs):
        #if POP_SIZE != len(x):
        #    print(f"WARNING: POP_SIZE != len(x) in evaluation step (this happens sometimes do not see why)\n"
        #          f"pop size: {POP_SIZE}; len x:{len(x)}")
        # Initialize
        dict_enemies = {}
        # Get fitness for each enemy
        for enemy in self.enemies:
            self.env.update_parameter('enemies', [enemy])

            dict_enemies[enemy] = []
            for individual_id in range(len(x)):
                dict_enemies[enemy].append(
                    simulation(self.env, x[individual_id], inverted_fitness=True, fitness_function=individual_gain))

        # Return fitness outputs for enemies
        objectives_fitness = {
            "objective_hard": [np.mean([dict_enemies[enemy_id][ind_id] for enemy_id in [1, 6]]) for ind_id in
                               range(len(x))],
            "objective_medium": [np.mean([dict_enemies[enemy_id][ind_id] for enemy_id in [2, 5, 8]]) for ind_id in
                                 range(len(x))],
            "objective_easy": [np.mean([dict_enemies[enemy_id][ind_id] for enemy_id in [3, 4, 7]]) for ind_id in
                               range(len(x))],
        }

        out["F"] = np.column_stack([objectives_fitness[key] for key in objectives_fitness.keys()])
        return out

In [5]:
env, n_genes = init_env('local_search_test', ENEMIES, n_hidden_neurons)
env.update_parameter('multiplemode', 'no')
problem = objectives(
        env=env,
        n_genes=n_genes,
        enemies=ENEMIES,
        n_objectives=3)

In [6]:
x_curr = solutions[[0],:]
x_best = np.copy(x_curr)

f_curr = problem._evaluate(x_curr, {})['F']
f_best = np.copy(f_curr)

In [33]:
# This is the function that needs to be defined, this is how you find neighbors of solution x
# For now, it gives a neighborhood where every gene has one 'mutation' or difference (the np.random.normal)
# I could not find clear examples of algorithms for the neighbor generating

def neighborhood(x, n):
    nbrhood = []
    for i in range(n):
        new_nbr = np.copy(x)
        new_nbr[0][i] = new_nbr[0][i] * np.random.normal()
        nbrhood.append(new_nbr)
    
    return nbrhood

In [8]:
done = False
solutionsChecked = 0

while not done:
    Neighborhood = neighborhood(x_curr)
    for s in Neighborhood:
        solutionsChecked += 1
        
        try:
            eval_s = problem._evaluate(s, {})['F']
        except:
            # print("Infeasible solution evaluated")
            continue
        
        if eval_s[0].mean() > f_best[0].mean():
            x_best = np.copy(s)
            f_best = np.copy(eval_s)
    
    if np.array_equal(f_best, f_curr):
        done = True
    else:
        x_curr = np.copy(x_best)
        f_curr = np.copy(f_best)
        
        print(f"Total number of solutions checked: {solutionsChecked}")
        print(f"Best value found so far: {f_best}")
        
print(f"Final number of solutions checked: {solutionsChecked}")
print(f"Best value found: {f_best}")
print(f"Best solution: {x_best}")

New neighborhood generated
New neighbor evaluated
New neighbor evaluated
New neighbor evaluated
New neighbor evaluated
New neighbor evaluated
Total number of solutions checked: 5
Best value found so far: [[0.00557209 0.00461264 0.00521065]]
New neighborhood generated
New neighbor evaluated
New neighbor evaluated
New neighbor evaluated
New neighbor evaluated
New neighbor evaluated
Total number of solutions checked: 10
Best value found so far: [[0.00590245 0.00479656 0.00521107]]
New neighborhood generated
New neighbor evaluated
New neighbor evaluated
New neighbor evaluated
New neighbor evaluated
New neighbor evaluated
Total number of solutions checked: 15
Best value found so far: [[0.00669833 0.00488379 0.00542057]]
New neighborhood generated
New neighbor evaluated
New neighbor evaluated
New neighbor evaluated
New neighbor evaluated
New neighbor evaluated
Total number of solutions checked: 20
Best value found so far: [[0.0069124  0.00482737 0.00632943]]
New neighborhood generated
New ne

In [30]:
x_curr = solutions[[0],:]
x_best = np.copy(x_curr)

f_curr = problem._evaluate(solutions, {})['F']
f_best = np.copy(f_curr)

print(f_curr)

pop size: 1; len x:23
[[0.00607702 0.00324837 0.00391076]
 [0.00557703 0.00327986 0.00394788]
 [0.00531855 0.00383785 0.00349158]
 [0.00390325 0.0033167  0.0045169 ]
 [0.00529449 0.0030659  0.00406615]
 [0.00543429 0.00314695 0.00406601]
 [0.00607702 0.00326428 0.00384838]
 [0.00544423 0.00383532 0.00347981]
 [0.00544423 0.00383532 0.00347981]
 [0.00503533 0.00324255 0.00418342]
 [0.0052944  0.00307404 0.00406615]
 [0.00607702 0.00326428 0.00384838]
 [0.00529449 0.0030659  0.00406615]
 [0.00503533 0.00324255 0.00418342]
 [0.0052947  0.00321195 0.00406601]
 [0.0054359  0.00310886 0.00406411]
 [0.00557703 0.00321795 0.00397778]
 [0.00607702 0.00326428 0.00384838]
 [0.00557703 0.00327986 0.00394788]
 [0.00557703 0.00321795 0.00397778]
 [0.00529449 0.0030659  0.00406615]
 [0.00543429 0.00314695 0.00406601]
 [0.00574189 0.00401288 0.00341699]]


In [34]:
t = 100000
M = 10
k = 0
n_nbrs = 25

solutionsChecked = 0

done = False
while not done:
    if t < 1:
        done = True
    
    m = 0
    while m < M:
        solutionsChecked += 1
        print("k = {}, m = {}, s = {} \n".format(k,m,solutionsChecked))
        
        N = neighborhood(x_curr, n_nbrs)
        # s = N[np.random.randint(len(N), size=M)]
        s = N[np.random.randint(len(N))]
        
        
        try:
            eval_s = problem._evaluate(s, {})['F']
        except Exception as e:
            break
        
        if eval_s[0].mean() >= f_best[0].mean():
            x_best = np.copy(s)
            f_best = np.copy(eval_s)
            
            f_solution.append(x_curr)
        else:
            p = np.exp(-(f_curr[0].mean() - eval_s[0].mean()) / t)
            test_p = np.random.uniform(0, 1)
            if test_p <= p:
                x_best = np.copy(s)
                f_best = np.copy(eval_s)
        
        m += 1
        
    k += 1
    t = 0.8*t
    print(f"Temperature lowered to {t}")
    
print(f"Final number of solutions checked: {solutionsChecked}")
print(f"Best solution: {x_best} (F: {f_best})")

k = 0, m = 0, s = 1 



TypeError: only integer scalar arrays can be converted to a scalar index